In [168]:
#https://github.com/Currie32/Chatbot-from-Movie-Dialogue/blob/master/Chatbot_Attention.ipynb

import pandas as pd
import csv 
import string 
import re
import tensorflow as tf
tweetdata = pd.read_csv("tweets.csv", sep='delimiter', engine='python')
retweetdata = pd.read_csv("retweets.csv", sep='delimiter', engine='python')
tf.__version__

'1.0.0'

In [169]:
#create dataframe of the tweets and retweets
df = pd.DataFrame(tweetdata)
df1 = pd.DataFrame(retweetdata)
tweets = df["Tweets"].tolist()
retweets = df1["Retweets"].tolist()

In [170]:
limit = 0
for i in range(limit, limit+5):
    print(tweets[i])
    print(retweets[i])
    print()

Something resembling hell': how does the rest of the world view the UK? https://t.co/mobKGxKBkH https://t.co/0AUuM3syKm
Communities Secretary @RobertJenrick has today announced £20m for councils to prepare for #Brexit and asked them each to appoint a Brexit lead to work with central government.

Don't agree with his #brexit view, but @DaveRowntree on @BBC6Music was a good listen. #festivalofthemoon #blur @StuartMaconie
"Jeremy Corbyn today:

@DmitryOpines @TheScepticIsle But I believe I will fly #brexit
'>If Johnson tries to force through No Deal we will put down a motion for a referendum on No Deal vs Remain, and will back for remain<

"Boris to employ 'Benny Hill Strategy' in Brexit negotiations
#brexit #LabourBrexitPolicy #Remain #ToryMess @jeremycorbyn  "

This morning, Downing Street has announced that Mr Johnson intends to utilise a 'Benny Hill Strategy' during the next round of Brexit negotiations in Brussels.  #Boris #BorisJohnson #brexit #  "
"Pro leave voter sends DEATH THREA

In [171]:
# Compare lengths of tweets and retweets
print(len(tweets))
print(len(retweets))

23403
32697


In [172]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.
        Also removing hashtags and @users
    '''

    text = text.lower()
    
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = re.sub(r"http\S+", "", text)
    
    return text

In [173]:
# Clean the data
clean_tweets = []
for tweet in tweets:
    clean_tweets.append(clean_text(tweet))
    
clean_retweets = []    
for retweet in retweets:
    clean_retweets.append(clean_text(retweet))

In [174]:
# Find the length of sentences
lengths = []
for tweets in clean_tweets:
    lengths.append(len(tweet.split()))
for retweet in clean_retweets:
    lengths.append(len(retweet.split()))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [175]:
lengths.describe()

,counts
count,56100.000000
mean,17.331569
std,10.863651
min,0.000000
25%,8.000000
50%,21.000000
75%,21.000000
max,99.000000


In [200]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.???
min_line_length = 1
max_line_length = 60

# Filter out the tweets that are too short/long
short_tweets_temp = []
short_retweets_temp = []

i = 0
for tweet in clean_tweets:
    if len(tweet.split()) >= min_line_length and len(tweet.split()) <= max_line_length:
        short_tweets_temp.append(tweet)
        short_retweets_temp.append(clean_retweets[i])
    i += 1

# Filter out the retweets that are too short/long
short_tweets = []
short_retweets = []

i = 0
for retweet in short_retweets_temp:
    if len(retweet.split()) >= min_line_length and len(retweet.split()) <= max_line_length:
        short_retweets.append(retweet)
        short_tweets.append(short_tweets_temp[i])
    i += 1

In [250]:
# Compare the number of lines we will use with the total number of lines.
print("# of tweets:", len(short_tweets))
print("# of retweets:", len(short_retweets))
print("% of data used: {}%".format(round(len(short_retweets)/len(retweets),4)*100))

# of tweets: 20166
# of retweets: 20166
% of data used: 61.68%


In [202]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for tweet in short_tweets:
    for word in tweet.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for retweet in short_retweets:
    for word in retweet.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [233]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 5
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [234]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 28576
Size of vocab we will use: 7856


In [235]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
tweets_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        tweets_vocab_to_int[word] = word_num
        word_num += 1
        
retweets_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        retweets_vocab_to_int[word] = word_num
        word_num += 1

In [236]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    tweets_vocab_to_int[code] = len(tweets_vocab_to_int)+1
    
for code in codes:
    retweets_vocab_to_int[code] = len(retweets_vocab_to_int)+1

In [237]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
tweets_int_to_vocab = {v_i: v for v, v_i in tweets_vocab_to_int.items()}
retweets_int_to_vocab = {v_i: v for v, v_i in retweets_vocab_to_int.items()} 

In [251]:
# Check the length of the dictionaries.
print(len(tweets_vocab_to_int))
print(len(tweets_int_to_vocab))
print(len(retweets_vocab_to_int))
print(len(retweets_int_to_vocab))

7860
7860
7860
7860


In [252]:
# Add the end of sentence token to the end of every retweet.
for i in range(len(short_retweets)):
    short_retweets[i] += ' <EOS>'

In [253]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
tweets_int = []
for tweet in short_tweets:
    ints = []
    for word in tweet.split():
        if word not in tweets_vocab_to_int:
            ints.append(tweets_vocab_to_int['<UNK>'])
        else:
            ints.append(tweets_vocab_to_int[word])
    tweets_int.append(ints)
    
retweets_int = []
for retweet in short_retweets:
    ints = []
    for word in retweet.split():
        if word not in retweets_vocab_to_int:
            ints.append(retweets_vocab_to_int['<UNK>'])
        else:
            ints.append(retweets_vocab_to_int[word])
    retweets_int.append(ints)

In [254]:
# Check the lengths
print(len(tweets_int))
print(len(retweets_int))

20166
20166


In [255]:
# Calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for tweet in tweets_int:
    for word in tweet:
        if word == tweets_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for retweet in retweets_int:
    for word in retweet:
        if word == retweets_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 799535
Number of times <UNK> is used: 37483
Percent of words that are <UNK>: 4.69%


In [256]:
# Sort tweets and retweets by the length of tweets.
# This will reduce the amount of padding during training
# Which should speed up training and help to reduce the loss

sorted_tweets = []
sorted_retweets = []

for length in range(1, max_line_length+1):
    for i in enumerate(tweets_int):
        if len(i[1]) == length:
            sorted_tweets.append(tweets_int[i[0]])
            sorted_retweets.append(retweets_int[i[0]])

print(len(sorted_tweets))
print(len(sorted_retweets))
print()
for i in range(3):
    print(sorted_tweets[i])
    print(sorted_retweets[i])
    print()

20166
20166

[4037]
[5142, 7858, 7858, 7858, 7858, 7858, 7858, 7858]

[3979]
[440, 6810, 1763, 7858, 7858, 7858, 7858, 7858, 7858, 7858]

[4169]
[912, 7227, 6810, 2374, 1069, 1926, 5789, 4479, 4590, 1922, 6147, 4085, 871, 1489, 5936, 5440, 7858, 7858, 7858, 7858, 7858, 7858, 7858]



In [63]:
def model_inputs():
    
    '''Create palceholders for inputs to the model.  A placeholder is simply a variable that we will assign data to at a later date. 
        It allows us to create our operations and build our computation graph, without needing the data. 
        Bassicly a place in memory where we will store value later on
        the arguments that it takes is the dtype of the data which will be put in, the shape of the tensor and the name for the operation
        
        Questions for Ben : Why do we specify the shape as [None, None] when having it blank will have as default a None shape?
        
        
        Answer: inputs placeholder will be fed with English sentence data, and its shape is [None, None]. The first None means the batch size, 
        and the batch size is unknown since user can set it. The second None means the lengths of sentences. 
        The maximum length of setence is different from batch to batch, so it cannot be set with the exact number.
        '''

    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [4]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the beginning of each batch
        tf.strided_slice() extracts a strided slice of a tensor.it will remove the final word from each batch
        tf.concat concatenates tensors along one dimension. tf.fill creates a tensor with a certain shape and fill it with 
        values. So here we have tensor with the shape batchsize and 1 witht he values vocab_to_int with "GO" as ending
        Benjamin: Explain the second line of code'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [6]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    
    '''Create the encoding layer
    tf.contrib contains contributed code. In this code we will use the rnn 
    which contains RNN Cells and additional RNN operations. BasicLSTMCell is for Basic LSTM recurrent network cell.
    the argument which will be passed in is the rnn_size.
    
    the DropoutWrapper adds dropout to inputs and outputs of the given cell. the arguments being firslty: input_keep_prob which
    is a unit Tensor or float between 0 and 1, input keep probability; if it is constant and 1, no input dropout will be added.
    Second one cells which we put lstm as they outperform GCU's.
    
    Thirdly we pass a RNN cell composed sequentially of multiple simple cells. The first arguments are the cells.
    This being the drop from before times the number of layers we'll pass in
    
    Fourthly, making the encoder bidirectional proved to be much more effective than a simple feed forward network.
    the arguments which it takes is cell_fw which is an instance of RNNCell, to be used for forward direction.
    the second argument being cell_bw which is an instance of RNNCell, to be used for backward direction.
    the sequence_lenght which is containing the actual lengths for each of the sequences in the batch. If not 
    provided, all batch entries are assumed to be full sequences.
    the inputs which are the RNN inputs. Lastly the dtype which is The data type for the initial states and expected output.
    
    We return only the encoder’s state because it is the input for our decoding layer. 
    Simply put, the weights of the encoding cells are what interest us.
    Why ltsm with basic vell and the end_cell with multi?
    '''

    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size) 
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [8]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data
    firstly, we create a a tensor with all elements set to zero.
    The model performs best when the attention states are set with zeros. 
    '''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [67]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

In [68]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits

In [69]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits

In [70]:
# Set the Hyperparameters
epochs = 50
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 200
decoding_embedding_size = 200
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [71]:
# Reset the graph to ensure that it is ready for training
import tensorflow as tf 

tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()
    
# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

# Create the training and inference logits
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(retweets_vocab_to_int), 
    len(tweets_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    tweets_vocab_to_int)

# Create a tensor for the inference logits, needed if loading a checkpoint version of the model
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

C:\Users\ising\Anaconda3Victor\envs\py354\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ising\Anaconda3Victor\envs\py354\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ising\Anaconda3Victor\envs\py354\lib\site-packages\tensorflow\python\framework\dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ising\Anaconda3Victor\envs\py354\lib\site-pack

In [72]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [73]:
import numpy as np
def batch_data(tweets, retweets, batch_size):
    """Batch tweets and retweets together"""
    for batch_i in range(0, len(tweets)//batch_size):
        start_i = batch_i * batch_size
        tweets_batch = tweets[start_i:start_i + batch_size]
        retweets_batch = retweets[start_i:start_i + batch_size]
        pad_tweets_batch = np.array(pad_sentence_batch(tweets_batch, tweets_vocab_to_int))
        pad_retweets_batch = np.array(pad_sentence_batch(retweets_batch, retweets_vocab_to_int))
        yield pad_tweets_batch, pad_retweets_batch

In [74]:
# Validate the training with 10% of the data
train_valid_split = int(len(sorted_tweets)*0.15)

# Split the tweets and retweets into training and validating data
train_tweets = sorted_tweets[train_valid_split:]
train_retweets = sorted_retweets[train_valid_split:]

valid_tweets = sorted_tweets[:train_valid_split]
valid_retweets = sorted_retweets[:train_valid_split]

print(len(train_tweets))
print(len(valid_tweets))

10086
1779


In [ ]:
import time

display_step = 100 # Check training loss after every 100 batches
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_tweets))//batch_size//2)-1 # Modulus for checking validation loss
total_train_loss = 0 # Record the training loss for each display step
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

checkpoint = "best_model.ckpt" 

sess.run(tf.global_variables_initializer())

for epoch_i in range(1, epochs+1):
    for batch_i, (tweets_batch, retweets_batch) in enumerate(
            batch_data(train_tweets, train_retweets, batch_size)):
        start_time = time.time()
        _, loss = sess.run(
            [train_op, cost],
            {input_data: tweets_batch,
             targets: retweets_batch,
             lr: learning_rate,
             sequence_length: retweets_batch.shape[1],
             keep_prob: keep_probability})

        total_train_loss += loss
        end_time = time.time()
        batch_time = end_time - start_time
        
        if batch_i % display_step == 0:
            print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                  .format(epoch_i,
                          epochs, 
                          batch_i, 
                          len(train_tweets) // batch_size, 
                          total_train_loss / display_step, 
                          batch_time*display_step))
            total_train_loss = 0

        if batch_i % validation_check == 0 and batch_i > 0:
            total_valid_loss = 0
            start_time = time.time()
            for batch_ii, (tweets_batch, retweets_batch) in \
                    enumerate(batch_data(valid_tweets, valid_retweets, batch_size)):
                valid_loss = sess.run(
                cost, {input_data: tweets_batch,
                       targets: retweets_batch,
                       lr: learning_rate,
                       sequence_length: retweets_batch.shape[1],
                       keep_prob: 1})
                total_valid_loss += valid_loss
            end_time = time.time()
            batch_time = end_time - start_time
            avg_valid_loss = total_valid_loss / (len(valid_tweets) / batch_size)
            print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))
            
            # Reduce learning rate, but not below its minimum value
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            summary_valid_loss.append(avg_valid_loss)
            if avg_valid_loss <= min(summary_valid_loss):
                print('New Record!') 
                stop_early = 0
                saver = tf.train.Saver() 
                saver.save(sess, checkpoint)

            else:
                print("No Improvement.")
                stop_early += 1
                if stop_early == stop:
                    break
    
    if stop_early == stop:
        print("Stopping Training.")
        break

In [ ]:
def tweet_to_seq(tweet, vocab_to_int):
    
    '''Prepare the tweet for the model'''
    
    tweet = clean_text(tweet)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in tweet.split()]

In [ ]:
# Create your own input question
#input_question = 'How are you?'

# Use a question from the data as your input
random = np.random.choice(len(short_tweets))
input_tweet = short_tweets[random]

#input own question
#input_question = "Do you like Theresa May?"
#input_tweet = tweet_to_seq(input_question, tweets_vocab_to_int)

# Prepare the question
input_tweet = tweet_to_seq(input_tweet, tweets_vocab_to_int)

# Pad the questions until it equals the max_line_length
input_tweet = input_tweet + [tweets_vocab_to_int["<PAD>"]] * (max_line_length - len(input_tweet))
# Add empty questions so the the input_data is the correct shape
batch_shell = np.zeros((batch_size, max_line_length))
# Set the first question to be out input question
batch_shell[0] = input_tweet    
    
# Run the model with the input question
retweet_logits = sess.run(inference_logits, {input_data: batch_shell, 
                                            keep_prob: 1.0})[0]

# Remove the padding from the Question and Answer
pad_q = tweets_vocab_to_int["<PAD>"]
pad_a = retweets_vocab_to_int["<PAD>"]

print('Question')
print('  Word Ids:      {}'.format([i for i in input_tweet if i != pad_q]))
print('  Input Words: {}'.format([tweets_int_to_vocab[i] for i in input_tweet if i != pad_q]))

print('\nAnswer')
print('  Word Ids:      {}'.format([i for i in np.argmax(retweet_logits, 1) if i != pad_a]))
print('  Response Words: {}'.format([retweets_int_to_vocab[i] for i in np.argmax(retweet_logits, 1) if i != pad_a]))